In [25]:
!pip install -q accelerate peft   transformers trl

In [26]:
!pip install -U bitsandbytes==0.45.5

In [27]:
device_map = {"": 0}

In [38]:
from transformers import TrainingArguments , pipeline
from trl import  SFTConfig, SFTTrainer

In [29]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "NousResearch/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    #target_modules=["q_proj", "v_proj"],
    lora_dropout=0.12,
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = prepare_model_for_kbit_training(model)
model = get_peft_model(base_model, lora_config)

In [31]:
from datasets import load_dataset

In [32]:
dataset_path="output.json"
dataset=load_dataset("json",data_files=dataset_path)

In [33]:
# @title Default title text
# Training arguments
training_arguments = SFTConfig(  ##
        output_dir="Llama-2-7b-chat-hf",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        fp16=True,
        packing=False,  ##
        max_seq_length=512,  ##
        dataset_text_field="question",  ##
)



In [34]:

trainer = SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        peft_config=lora_config,
        args=training_arguments,
        processing_class=tokenizer,
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=83, training_loss=1.5600008677287274, metrics={'train_runtime': 218.5715, 'train_samples_per_second': 3.02, 'train_steps_per_second': 0.38, 'total_flos': 2120460567773184.0, 'train_loss': 1.5600008677287274})

In [37]:
new_model = "Llama-2-7b-chat-finetune"
trainer.model.save_pretrained(new_model)


In [46]:
from transformers import pipeline

prompt = "What are the risks of operating the drone in rainy weather?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

result = pipe(f"<s>[INST] {prompt} [/INST]")

# Extract generated text
generated_text = result[0]['generated_text']

# Remove the instruction part and keep only the model's answer
answer = generated_text.split("[/INST]")[-1].strip()

print(answer)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Operating the drone in rainy weather poses a risk of water damage to the drone and the electrical components, and may also result in reduced performance.


In [43]:
generated_text

"<s>[INST] What is the purpose of the position adjustment feature in the Spider-I UAV? [/INST] The position adjustment feature in the Spider-I UAV allows users to adjust the drone's position, providing flexibility and convenience in flight."

In [44]:
answer

"The position adjustment feature in the Spider-I UAV allows users to adjust the drone's position, providing flexibility and convenience in flight."

In [47]:

#the error related  to this The model 'PeftModelForCausalLM' is not supported for text-generation.
from transformers import AutoTokenizer
import torch

prompt = "What are the risks of operating the drone in rainy weather?"
input_text = f"<s>[INST] {prompt} [/INST]"

# Tokenize
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

# Decode and extract only the answer
decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)
answer = decoded.split("[/INST]")[-1].strip()

print(answer)


Operating the drone in rainy weather is a high-risk activity, as it may cause the drone to lose control, crash, or damage the equipment.</s>


In [35]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question'],
        num_rows: 660
    })
})